# SMA讀書會HW4-2

- 組別: 第 6 組
- 日期: 2024/5/28
- 資料來源: PTT 汽車版 2023.4~2024.4 一年 共15741 筆資料
- 分析目的:<br>
  1.以老師指導的碩論文為例，實作RAG應用.<BR>
  2.希望從過去一年的PTT 汽車版的討論內容進行社會網絡分析

# 一. Retrieve QA (RAG)

In [19]:
#! pip install langchain_community
#! pip install langchain_chroma
#! pip install install pypdf
#! pip install sentence-transformers
! pip install grandalf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.9 MB/s eta 0:00:00


In [5]:
# 載入必要套件
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_chroma import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.llms import HuggingFaceEndpoint
from getpass import getpass
import os

In [11]:
import os
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [12]:
os.chdir('/content/drive/MyDrive/data')

In [6]:
from google.colab import userdata
#HUGGINGFACEHUB_API_TOKEN = getpass()
HUGGINGFACEHUB_API_TOKEN = userdata.get('hf_access_token')
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN



In [7]:
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-1.1-7b-it", temperature=1.0,model_kwargs={'token':'HUGGINGFACEHUB_API_TOKEN', 'use_cache':False}
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEndpoint` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the from langchain-huggingface package and should be used instead. To use it run `pip install -U from langchain-huggingface` and import as `from from langchain_huggingface import llms import HuggingFaceEndpoint`.
  warn_deprecated(


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [13]:
# 載入論文:以文字探勘技術探討影響清潔用品牌推薦的關鍵因素(劉祐維)
loader = PyPDFLoader("etd-0704122-121350.pdf")
pages = loader.load()
pages
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
texts = text_splitter.split_documents(pages)
texts[:10]

[Document(page_content='國立中山大學資訊管理 學系 \n碩士論文  \nDepartment of Information Management  \nNational Sun Yat -sen University  \nMaster ’s Thesis  \n \n以文字探勘技術探討影響清潔用品牌推薦的關鍵因素  \n \nUsing Text Mining  Techniques to Identify  the Key Factors for \nRecommending C leaning Brands  \n \n \n研究生： 劉祐維  \n           Yu-Wei Liu  \n指導教授： 黃三益  博士  \nDr. San-Yih Hwang  \n \n中華民國 111年8月 \nAugust  2022', metadata={'source': 'etd-0704122-121350.pdf', 'page': 0}),
 Document(page_content='i \n 論文審定書', metadata={'source': 'etd-0704122-121350.pdf', 'page': 1}),
 Document(page_content='ii \n 誌 謝 \n感謝這兩年來一起修課的同學子平、家逸、明信、沛芸、志豪， 這段時間 大\n家一起討論作業，聚在子平家一起討論 報告、玩遊戲、吃飯聊天，彷彿又回到以\n前大學的時光，對於工作已有 10年的上班族來說是很棒的生活調劑，感謝夥伴\n們在求學路上 的相互幫忙 與鼓勵，讓這段學習旅程充實而愉快。  \n最重要的是感謝黃三益老師的細心指導，老師循序善誘的指導方式讓我受益\n良多，在 研修過社群媒體分析及資料庫系統課程後，知道老師想給學生的不僅有\n書本的內容，更重要的是實務的應用及思考能力，感謝老師的鼓勵讓我能在最後\n的時刻將論文完成，另外也感謝 31實驗室的協助，尤其是張家銘學長的在程式\n與想法的幫忙，讓我在數據的處理及分析上順暢許多，謝謝一同在論文口試上努\n力的彥威學長，能加入三益老師的實驗室是一件幸運的事。  \n謝謝我人生的好夥伴 文敏，帶這我上山下海，體驗許多以前未曾嘗試過的滑\n雪、滑水、登山、浮潛，感謝她豐富了我的生活，也

In [16]:

embedding_function = SentenceTransformerEmbeddings(model_name="lier007/xiaobu-embedding")

vectorstore = Chroma.from_documents(documents=texts, embedding=embedding_function)

retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/26.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

In [17]:
# query it
query = "研究目的是什麼？"
docs = vectorstore.similarity_search(query)

# print results
print(docs[0].page_content)

表 次................................ ................................ ................................ ...........................  viii 
第一章  緒論 ................................ ................................ ................................ ..................  1 
1-1 研究背景  ................................ ................................ ................................ ........  1 
1-2 研究動機  ................................ ................................ ................................ ........  2 
1-3 研究目的  ................................ ................................ ................................ ........  2 
1-4 論文結構  ................................ ................................ ................................ ........  3 
第二章  文獻探討 ................................ ................................ ................................ ..........  4


In [20]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_prompt_templete = """<start_of_turn>user
"角色": "研究生",
"服務內容":
你是一個樂於幫助學生的機器人，關於論文的問題你都可以依照以下文件中的規定來回答學生。\n
現在給予以下相關文件, 你將會基於以下 [文件][文件] 中的相關文件與 [回答規則][回答規則] 中的回答規則來回答學生的問題。\n
\n
[文件]
{context}
[文件]
\n
[回答規則]
只依據 [文件][文件] 中提供的資訊來回答，並且遵照下列的幾點規則：
- 不撒謊或幻想 [文件][文件] 中未明確提供的答案
- 如果不確定答案或答案未明確包含在 [文件][文件] 中，請回答：“我很抱歉，我不知道如何提供幫助。”
- 回答簡短、相關且簡潔的答案
- 回答字數請少於 150 個字
- 請在 [結果]: 之後回答你的答案
[回答規則]

<</SYS>>
現在，基於上述的文件與規則，回答以下 () 中的問題\n[學生]:({question})\n<end_of_turn>
<start_of_turn>model[結果]:"""

rag_prompt = PromptTemplate.from_template(rag_prompt_templete)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)
rag_chain.get_graph().print_ascii()

              +---------------------------------+           
              | Parallel<context,question>Input |           
              +---------------------------------+           
                    ****                ****                
                 ***                        ***             
               **                              ***          
+----------------------+                          **        
| VectorStoreRetriever |                           *        
+----------------------+                           *        
            *                                      *        
            *                                      *        
            *                                      *        
+---------------------+                     +-------------+ 
| Lambda(format_docs) |                     | Passthrough | 
+---------------------+                     +-------------+ 
                    ****                ****                
                        

In [21]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt_template = PromptTemplate.from_template(template)

In [25]:
rag_chain.invoke("研究動機是什麼？")

' 研究動機是為了找出影響品牌推薦度最顯著的構面。'

In [26]:
rag_chain.invoke("研究目的是什麼？")

' 研究目的是利用文字探勘技術推測滿意度，並找出影響滿意度最顯著的構面。'

In [24]:
rag_chain.invoke("研究方法是什麼？")

' 研究方法是利用文字探勘的方式，資料的來源是 PTT論壇，討論版則是以家事版為主展開到其他七個討論版。'

In [28]:
rag_chain.invoke("研究模型是什麼？")

' 研究模型架構圖顯示，滿意度由知覺品質與體驗價值兩個面向所組成。知覺品質包含產品的價格、品牌知名度與製造商的名聲，而體驗價值則為顧客對產品的印象。'

In [27]:
rag_chain.invoke("研究結論是什麼？")

' 根據回歸分析結果，外型、廣告和思考對滿意度有顯著影響。'

In [29]:
rag_chain.invoke("研究限制以及未來展望是什麼？")

' 研究限制包括資料集的局限性，例如文章來源的廣度和深度，以及情緒分析和主題模型的準確性。未來展望可以擴展資料集以涵蓋更多來源和形式的文章，並探索更先進的情緒分析和主題模型技術。'

# 二. 社會網絡分析

In [5]:
#導入必要套件
import jieba
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LdaMulticore
import time
import numpy as np
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.matutils import corpus2csc, corpus2dense, Sparse2Corpus
import matplotlib.pyplot as plt

In [9]:
#載入先前使用 transformers 處理NER 之csv結果
ner_results_df = pd.read_csv('car_ner_results.csv')
ner_results_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


sentence  \
0    以下為建議資訊車輛廠牌年份型號福特車輛原廠編成等級最頂級是否含保險否是否含領牌稅金否成交價格...   
1    大家晚安最近有觀察一些測試的一些影片感覺一些複雜路道車多的狀況有被改善許多的樣子覺得應該可以...   
2    引述之銘言影片連結起因是放棄人生號在雙黃線違規迴轉這位凱旋騎士跟著違規迴轉之後暴打放棄人生號...   
3    就是把馬路當賽道呀尤其是轉彎的時候出彎反而重踩油門所以當發現有人走在斑馬線上時根本來不及剎車...   
4    原文恕刪施主若從考慮安全的角度出發來討論這件事我舉雙手贊成但同時也請好好思考一件事每次在切外...   
..                                                 ...   
995  引述軍之銘言是這樣的今天到別人宿舍訪客整個停車場都沒有停車位看了一下剩下電動車位可以停到底電...   
996  抱歉更正一下這個影片講的是版本因為有推文說兩者價差不到萬所以確認了一下官方定價萬純油萬油電跟...   
997  引述秋風孤劍李鴻章之銘言檢舉他根本檢舉不完這是真的檢舉不完還未必全部受理成功有些人就是跟你嗆...   
998  是不是在全國都有第三人保險的前提下車體險就作用不大了一個性質是責任險一個性質是財物保險第一個...   
999  原文連結原文內容高雄市三民區今日上午發生一起車禍一部喜美轎車左轉時擦撞到對向要右轉的勞斯萊斯...   

                                       packed_sentence  \
0    以下(Ncd)　為(P)　建議(VE)　資訊(Na)　車輛(Na)　廠牌(Na)　年份(Na...   
1    大家(Nh)　晚安(VH)　最近(Nd)　有(V_2)　觀察(VE)　一些(Neqa)　測試...   
2    引述(VC)　之(DE)　銘言(Na)　影片(Na)　連結(VC)　起因(Na)　是(SHI...   
3    就是(D)　把(P)　馬路(Na)　當(VG)　賽道(Na)　呀(T)　尤其(D)　是(SH...   
4    原文(A)　恕(VL)　刪(VC)　施主(Na)　若(Cbb)　從(P)　考慮(VE)　安全...   
..                                                 ...   
995  引述(VC)　軍之(Na)　銘言(Na)　是(SHI)　這樣(VH)　的(DE)　今天(Nd...   
996  抱歉(VK)　更正(VC)　一下(Nd)　這(Nep)　個(Nf)　影片(Na)　講(VE)...   
997  引述(VC)　秋風孤劍(Na)　李鴻章(Nb)　之(DE)　銘言(Na)　檢舉(VC)　他(...   
998  是(SHI)　不(D)　是(SHI)　在(P)　全(Neqa)　國(Nc)　都(D)　有(V...   
999  原文(Na)　連結(VC)　原文(Na)　內容(Na)　高雄市(Nc)　三民區(Nc)　今日...   

                                              entities  
0    ["NerToken(word='三', ner='CARDINAL', idx=(125,...  
1    ["NerToken(word='台灣', ner='GPE', idx=(48, 50))...  
2    ["NerToken(word='四', ner='CARDINAL', idx=(183,...  
3    ["NerToken(word='今天', ner='DATE', idx=(138, 14...  
4    ["NerToken(word='三十公尺', ner='QUANTITY', idx=(1...  
..                                                 ...  
995  ["NerToken(word='今天', ner='DATE', idx=(10, 12)...  
996  ["NerToken(word='兩', ner='CARDINAL', idx=(21, ...  
997  ["NerToken(word='李鴻章', ner='PERSON', idx=(6, 9...  
998  ["NerToken(word='第三', ner='ORDINAL', idx=(8, 1...  
999  ["NerToken(word='高雄市', ner='GPE', idx=(8, 11))...  

[1000 rows x 3 columns]

In [10]:
# 只取專有名詞(Nb)之word 進行分析
ner_results_df['word'] = ner_results_df['packed_sentence'].apply(lambda x: ' '.join(word for word in x.split() if word[-4:] == "(Nb)"))
ner_results_df['word'] = ner_results_df['word'].replace("\(Nb\)", "", regex=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:3: DeprecationWarning: invalid escape sequence '\('
<>:3: DeprecationWarning: invalid escape sequence '\('
<ipython-input-10-200b7467f7c5>:3: DeprecationWarning: invalid escape sequence '\('
  ner_results_df['word'] = ner_results_df['word'].replace("\(Nb\)", "", regex=True)


In [11]:
ner_results_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


sentence  \
0    以下為建議資訊車輛廠牌年份型號福特車輛原廠編成等級最頂級是否含保險否是否含領牌稅金否成交價格...   
1    大家晚安最近有觀察一些測試的一些影片感覺一些複雜路道車多的狀況有被改善許多的樣子覺得應該可以...   
2    引述之銘言影片連結起因是放棄人生號在雙黃線違規迴轉這位凱旋騎士跟著違規迴轉之後暴打放棄人生號...   
3    就是把馬路當賽道呀尤其是轉彎的時候出彎反而重踩油門所以當發現有人走在斑馬線上時根本來不及剎車...   
4    原文恕刪施主若從考慮安全的角度出發來討論這件事我舉雙手贊成但同時也請好好思考一件事每次在切外...   
..                                                 ...   
995  引述軍之銘言是這樣的今天到別人宿舍訪客整個停車場都沒有停車位看了一下剩下電動車位可以停到底電...   
996  抱歉更正一下這個影片講的是版本因為有推文說兩者價差不到萬所以確認了一下官方定價萬純油萬油電跟...   
997  引述秋風孤劍李鴻章之銘言檢舉他根本檢舉不完這是真的檢舉不完還未必全部受理成功有些人就是跟你嗆...   
998  是不是在全國都有第三人保險的前提下車體險就作用不大了一個性質是責任險一個性質是財物保險第一個...   
999  原文連結原文內容高雄市三民區今日上午發生一起車禍一部喜美轎車左轉時擦撞到對向要右轉的勞斯萊斯...   

                                       packed_sentence  \
0    以下(Ncd)　為(P)　建議(VE)　資訊(Na)　車輛(Na)　廠牌(Na)　年份(Na...   
1    大家(Nh)　晚安(VH)　最近(Nd)　有(V_2)　觀察(VE)　一些(Neqa)　測試...   
2    引述(VC)　之(DE)　銘言(Na)　影片(Na)　連結(VC)　起因(Na)　是(SHI...   
3    就是(D)　把(P)　馬路(Na)　當(VG)　賽道(Na)　呀(T)　尤其(D)　是(SH...   
4    原文(A)　恕(VL)　刪(VC)　施主(Na)　若(Cbb)　從(P)　考慮(VE)　安全...   
..                                                 ...   
995  引述(VC)　軍之(Na)　銘言(Na)　是(SHI)　這樣(VH)　的(DE)　今天(Nd...   
996  抱歉(VK)　更正(VC)　一下(Nd)　這(Nep)　個(Nf)　影片(Na)　講(VE)...   
997  引述(VC)　秋風孤劍(Na)　李鴻章(Nb)　之(DE)　銘言(Na)　檢舉(VC)　他(...   
998  是(SHI)　不(D)　是(SHI)　在(P)　全(Neqa)　國(Nc)　都(D)　有(V...   
999  原文(Na)　連結(VC)　原文(Na)　內容(Na)　高雄市(Nc)　三民區(Nc)　今日...   

                                              entities  \
0    ["NerToken(word='三', ner='CARDINAL', idx=(125,...   
1    ["NerToken(word='台灣', ner='GPE', idx=(48, 50))...   
2    ["NerToken(word='四', ner='CARDINAL', idx=(183,...   
3    ["NerToken(word='今天', ner='DATE', idx=(138, 14...   
4    ["NerToken(word='三十公尺', ner='QUANTITY', idx=(1...   
..                                                 ...   
995  ["NerToken(word='今天', ner='DATE', idx=(10, 12)...   
996  ["NerToken(word='兩', ner='CARDINAL', idx=(21, ...   
997  ["NerToken(word='李鴻章', ner='PERSON', idx=(6, 9...   
998  ["NerToken(word='第三', ner='ORDINAL', idx=(8, 1...   
999  ["NerToken(word='高雄市', ner='GPE', idx=(8, 11))...   

                                                  word  
0                                                   福特  
1                                                  特斯拉  
2                                                       
3                                                       
4                                                       
..                                                 ...  
995                                              賓士 賓士  
996                                                     
997                                                李鴻章  
998                                                     
999  喜美 勞斯萊斯 曾姓 勞斯萊斯 本田 喜美 喜美 曾姓 本田 黃姓 勞斯萊斯 喜美 黃姓 勞...  

[1000 rows x 4 columns]

In [26]:
from sklearn.feature_extraction.text import CountVectorizer
import pyvis
import names
import numpy as np
import random
import IPython

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [17]:
# 這邊採用助教提供之 matrix function 建立分析圖
def random_color():
  r = lambda: random.randint(0,255)
  return '#%02X%02X%02X' % (r(),r(),r())
def matPresentGraph(mat:np.array, node_id:list, node_type:list = None, node_value:list = None, directed = True, edge_color = None) -> pyvis.network.Network:
  '''
    根據輸入的matrix建立網路圖
    mat: adjacency matrix
    node_id: 要設定的節點id
    node_type: 用來區分不同類型的節點（例如：不同群組），會作為節點的title
    node_value: 節點大小
    directed: 有向/無向
    edge_color: edge顏色

  '''

  # 如果設定為無向圖，則檢查輸入的 matrix 是否對稱
  if directed != True:
    if (mat == mat.transpose()).all():
      print('matrix is Symmetric')
    else:
      print('matrix is not Symmetric')

  # 若無設定 node_type，則為所有節點設置相同的顏色(隨機)和類型
  if node_type == None:
    c = random_color()
    node_colors = [c for i in range(len(node_id))]
    node_type = [" " for i in range(len(node_id))]

  else: # 若有設定node_type，則為每種類型分配一個隨機顏色
    node_color_map = {}

    for i in set(node_type):
      while True:
        c = random_color()
        if c not in node_color_map.values():
          break
      node_color_map[i] = c

    node_colors = [node_color_map[i] for i in node_type]

  # 若無設定edge_color，則設置隨機顏色
  if edge_color == None:
    edge_color = random_color()
  # edge_color = random_color()

  # 若無設定node_value，則設定所有節點的大小為1
  if node_value == None:
    node_value = [1 for i in range(len(node_id))]

  net = pyvis.network.Network(notebook=True, directed = directed, cdn_resources='in_line')

  # 加入點
  net.add_nodes(
      nodes = node_id,
      value = node_value,
      label = node_id,
      title = node_type,
      color = node_colors
  )

  # 加入邊
  for row in range(len(node_id)):
    for col in range(len(node_id)):
      if mat[row][col] > 0.:
        net.add_edge(
            node_id[row], node_id[col], width = mat[row][col], color = edge_color, title = mat[row][col]
        )

  return net

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
# 取 30 個重要字
vectorizer = CountVectorizer(min_df = 3, max_features =30)
X = vectorizer.fit_transform(ner_results_df["word"])
vocabulary = vectorizer.get_feature_names_out()

# 轉成dataframe
DTM_df = pd.DataFrame(columns = vocabulary, data = X.toarray())
DTM_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


三菱  上座部  佛教  六和  勞斯萊斯  和泰  喜美  國泰  大乘  本田  ...  裕隆  豐田  賓士  賴男  鄭姓  鄭男  \
0     0    0   0   0     0   0   0   0   0   0  ...   0   0   0   0   0   0   
1     0    0   0   0     0   0   0   0   0   0  ...   0   0   0   0   0   0   
2     0    0   0   0     0   0   0   0   0   0  ...   0   0   0   0   0   0   
3     0    0   0   0     0   0   0   0   0   0  ...   0   0   0   0   0   0   
4     0    0   0   0     0   0   0   0   0   0  ...   0   0   0   0   0   0   
..   ..  ...  ..  ..   ...  ..  ..  ..  ..  ..  ...  ..  ..  ..  ..  ..  ..   
995   0    0   0   0     0   0   0   0   0   0  ...   0   0   2   0   0   0   
996   0    0   0   0     0   0   0   0   0   0  ...   0   0   0   0   0   0   
997   0    0   0   0     0   0   0   0   0   0  ...   0   0   0   0   0   0   
998   0    0   0   0     0   0   0   0   0   0  ...   0   0   0   0   0   0   
999   0    0   0   0     4   0   4   0   0   2  ...   0   0   0   0   0   0   

     陳姓  陳男  馬斯克  高姓  
0     0   0    0   0  
1     0   0    0   0  
2     0   0    0   0  
3     0   0    0   0  
4     0   0    0   0  
..   ..  ..  ...  ..  
995   0   0    0   0  
996   0   0    0   0  
997   0   0    0   0  
998   0   0    0   0  
999   0   0    0   0  

[1000 rows x 30 columns]

In [29]:
# 計算詞之間的相關係數
corr_matrix = np.corrcoef(DTM_df.T)
# 轉成dataframe
Cor_df = pd.DataFrame(corr_matrix, index = DTM_df.columns, columns = DTM_df.columns)
Cor_df.reset_index(inplace = True, drop = True)
Cor_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


三菱       上座部        佛教        六和      勞斯萊斯        和泰        喜美  \
0   1.000000 -0.006663 -0.006767 -0.005520 -0.004683 -0.009923  0.073112   
1  -0.006663  1.000000  0.984642 -0.004848 -0.004113 -0.008715 -0.005991   
2  -0.006767  0.984642  1.000000 -0.004924 -0.004177 -0.008851 -0.006084   
3  -0.005520 -0.004848 -0.004924  1.000000 -0.003407  0.266245 -0.004963   
4  -0.004683 -0.004113 -0.004177 -0.003407  1.000000 -0.006125  0.296532   
5  -0.009923 -0.008715 -0.008851  0.266245 -0.006125  1.000000  0.007975   
6   0.073112 -0.005991 -0.006084 -0.004963  0.296532  0.007975  1.000000   
7  -0.005215 -0.004580 -0.004651 -0.003794 -0.003219 -0.006820 -0.004688   
8  -0.006750  0.985263  0.997494 -0.004911 -0.004167 -0.008829 -0.006069   
9   0.025049 -0.009250 -0.009395  0.057057  0.093903  0.042634  0.278447   
10 -0.006767 -0.005944 -0.006036 -0.004924 -0.004177 -0.008851 -0.006084   
11 -0.007354 -0.006459 -0.006560 -0.005351 -0.004540 -0.009619 -0.006612   
12 -0.007214 -0.006336 -0.006435 -0.005249 -0.004453 -0.009435 -0.006486   
13 -0.003275  0.446560  0.402809 -0.002383 -0.002022 -0.004283 -0.002944   
14 -0.006112  0.138422  0.124355 -0.004447 -0.003773 -0.007994 -0.005495   
15 -0.015398  0.018223  0.029254 -0.011203 -0.009505 -0.008679 -0.013844   
16  0.044488 -0.006150 -0.006246  0.088563 -0.004323  0.023493 -0.006296   
17 -0.007634 -0.006705 -0.006810  0.081234 -0.004712  0.048849 -0.006864   
18 -0.005577 -0.004898 -0.004975 -0.004058 -0.003443 -0.007295 -0.005014   
19 -0.005471 -0.004805 -0.004880 -0.003981 -0.003377 -0.007156 -0.004919   
20 -0.005912 -0.005192 -0.005273  0.278489 -0.003649  0.028783 -0.005315   
21 -0.011544  0.098885  0.137336  0.618165 -0.007126  0.265333 -0.010379   
22 -0.009010  0.264122  0.237544 -0.006556  0.007681 -0.011785 -0.008101   
23 -0.005497 -0.004828 -0.004903 -0.003999 -0.003393 -0.007190 -0.004942   
24 -0.004998 -0.004390 -0.004458 -0.003637 -0.003085 -0.006537 -0.004494   
25 -0.005348 -0.004697 -0.004770 -0.003891 -0.003301 -0.006995 -0.004808   
26 -0.007827  0.259790  0.233751 -0.005694 -0.004831 -0.010237 -0.007037   
27 -0.006470  0.349486  0.314859 -0.004708 -0.003994 -0.008463 -0.005817   
28 -0.004413 -0.003876 -0.003936 -0.003211 -0.002724 -0.005772 -0.003968   
29 -0.004142 -0.003638 -0.003695 -0.003014 -0.002557 -0.005417 -0.003724   

          國泰        大乘        本田  ...        裕隆        豐田        賓士        賴男  \
0  -0.005215 -0.006750  0.025049  ... -0.005912 -0.011544 -0.009010 -0.005497   
1  -0.004580  0.985263 -0.009250  ... -0.005192  0.098885  0.264122 -0.004828   
2  -0.004651  0.997494 -0.009395  ... -0.005273  0.137336  0.237544 -0.004903   
3  -0.003794 -0.004911  0.057057  ...  0.278489  0.618165 -0.006556 -0.003999   
4  -0.003219 -0.004167  0.093903  ... -0.003649 -0.007126  0.007681 -0.003393   
5  -0.006820 -0.008829  0.042634  ...  0.028783  0.265333 -0.011785 -0.007190   
6  -0.004688 -0.006069  0.278447  ... -0.005315 -0.010379 -0.008101 -0.004942   
7   1.000000 -0.004640 -0.007239  ... -0.004063 -0.007935 -0.006193 -0.003778   
8  -0.004640  1.000000 -0.009371  ... -0.005260  0.144355  0.212453 -0.004891   
9  -0.007239 -0.009371  1.000000  ...  0.006196  0.156301 -0.012509 -0.007631   
10 -0.004651 -0.006021 -0.009395  ... -0.005273 -0.010297 -0.008037 -0.004903   
11 -0.005055 -0.006543 -0.010209  ... -0.005731 -0.011191 -0.008734 -0.005329   
12 -0.004959 -0.006419 -0.010015  ... -0.005621 -0.010977 -0.008568 -0.005227   
13 -0.002251  0.422036 -0.004547  ... -0.002552 -0.004983 -0.003890 -0.002373   
14 -0.004201  0.130518 -0.008485  ... -0.004763  0.004986 -0.007259 -0.004429   
15  0.078858  0.024893 -0.005102  ... -0.011267  0.014420 -0.001116  0.261772   
16 -0.004813 -0.006231  0.005734  ...  0.011218  0.157842 -0.008317 -0.005074   
17 -0.005247 -0.006793  0.005315  ... -0.005949  0.040430  0.041305 -0.005531   
18 -0.003833 -0.004962 -0.007743  ... -0.004346 -0.008487 -0.006624 -0.004041   
19 -0.003761 -0.004868 -0.007595 

In [30]:
# 讓對角等於0
mat = Cor_df.values
di = [i for i in range(mat.shape[0])]
mat[di,di] = 0
mat[mat<0.1] = 0
mat = mat*10
mat

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 9.84641879, 0.        , 0.        ,
        0.        , 0.        , 0.        , 9.85262542, 0.        ,
        0.        , 0.        , 0.        , 4.46560477, 1.38421828,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 2.6412242 , 0.        , 0.        ,
        0.        , 2.59790275, 3.49486276, 0.        , 0.        ],
       [0.        , 9.84641879, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 9.97494358, 0.        ,
        0.        , 0.        , 0.        , 4.

### 從 下圖 network.html 中 可以看到右上角一個"特斯拉"與"賴男"\有高相關連結，這是因為去年的 特斯拉掉天坑事件，賴男是租賃車，但是因為建築工地路旁塌陷關係，車子掉落但初期保險公司不理賠，租車公司又要求清償的事件
https://polida.com.tw/post/car-damage-insurance

In [31]:
#產生社會網絡分析圖
org_list = vocabulary
net = matPresentGraph(mat,org_list,directed = False)
net.show("./netWork.html")
#os.system("start netWork.html")
IPython.display.HTML('netWork.html')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


matrix is not Symmetric
./netWork.html
